# Estudo de LangGraph: Do Workflow Linear ao Condicional

**Tutor:** Gemini
**Aluno:** Fábio

---

### Objetivo deste Estudo

Este notebook é um guia prático e didático para entender os conceitos fundamentais do **LangGraph**. Nosso objetivo é evoluir de um agente com um fluxo de trabalho simples e linear para um agente mais inteligente, capaz de tomar decisões e seguir caminhos diferentes com base na entrada do usuário.

Para isso, vamos construir dois agentes:

1.  **O Assistente de Perguntas:** Um agente com um fluxo **linear** de duas etapas para otimizar e responder perguntas.
2.  **O Agente de Viagens Inteligente:** Um agente com um fluxo **condicional**, que atua como um roteador para chamar ferramentas especializadas.

Seguiremos um **framework de trabalho** estruturado em 4 fases: Conceito, Estrutura, Implementação e Validação.

## Parte 1: O Workflow Linear - O "Assistente de Perguntas"

Neste primeiro exercício, construiremos um agente que executa um processo de duas etapas sequenciais. É um exemplo perfeito para entender os conceitos básicos de **Estado**, **Nós** e **Arestas**.

### Fase 1: ✍️ Conceito e Arquitetura

**🎯 Missão:** Criar um agente que primeiro **reformula a pergunta** do usuário para ser mais clara e, em seguida, **responde** a essa nova pergunta, agindo como um tutor de Python.

**🗺️ Fluxograma:**
`START` -> `[Nó de Reformulação]` -> `[Nó de Resposta]` -> `END`

### Fase 2: 🏛️ Estrutura (Estado e Ferramentas)

Primeiro, importamos as bibliotecas e definimos o **Estado**, que é a memória compartilhada do nosso agente.

In [ ]:
import os
from typing import TypedDict

from dotenv import load_dotenv
from IPython.display import Image, display
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI
from langgraph.graph import END, START, StateGraph

# Carrega as variáveis de ambiente (ex: OPENAI_API_KEY)
load_dotenv()

# Configuração do LLM que será usado pelos nós
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.0)


# Definição do Estado
class AgentStateLinear(TypedDict):
    """Define a estrutura de dados para o Assistente de Perguntas."""
    original_question: str
    question_clarified: str
    answer: str

### Fase 3: ⚙️ Implementação (Nós e Grafo)

Agora, codificamos os **Nós** (as funções que realizam o trabalho) e montamos o **Grafo**.

In [ ]:
# Definição dos Nós
def reformulate_node(state: AgentStateLinear) -> dict:
    """Nó 1: Pega a pergunta original e a reformula para ser mais clara."""
    print("--- (Nó 1: Reformulando Pergunta) ---")
    original_question = state["original_question"]
    
    prompt = [
        SystemMessage("Sua tarefa é reformular a pergunta do usuário para que ela seja mais clara e direta. Retorne apenas a pergunta reformulada."),
        HumanMessage(original_question)
    ]
    
    response = llm.invoke(prompt)
    reformulated = response.content
    
    return {"question_clarified": reformulated}

def answer_node(state: AgentStateLinear) -> dict:
    """Nó 2: Pega a pergunta reformulada e a responde como um tutor de Python."""
    print("--- (Nó 2: Gerando Resposta) ---")
    reformulated_question = state["question_clarified"]
    
    prompt = [
        SystemMessage("Você é um tutor expert em Python. Responda de forma clara, didática e com exemplos de código."),
        HumanMessage(reformulated_question)
    ]
    
    response = llm.invoke(prompt)
    answer = response.content
    
    return {"answer": answer}

In [ ]:
# Montagem do Grafo
workflow_linear = StateGraph(AgentStateLinear)

workflow_linear.add_node("reformulate", reformulate_node)
workflow_linear.add_node("answer", answer_node)

# Define as arestas (o caminho)
workflow_linear.set_entry_point("reformulate")
workflow_linear.add_edge("reformulate", "answer")
workflow_linear.add_edge("answer", END)

# Compilação
agent_linear = workflow_linear.compile()

### Fase 4: ✅ Validação (Execução e Análise)

Finalmente, visualizamos o grafo para confirmar se a arquitetura está correta e executamos o agente.

In [ ]:
# Visualização do Grafo
print("Arquitetura do Agente Linear:")
display(Image(agent_linear.get_graph().draw_mermaid_png()))

In [ ]:
# Execução
pergunta_usuario = "como eu faço pra pegar só os numero par numa lista em python?"
initial_state = {"original_question": pergunta_usuario}

final_state = agent_linear.invoke(initial_state)

print("\n--- RESULTADO FINAL ---")
print(f"Pergunta Original: {final_state['original_question']}")
print(f"Pergunta Reformulada: {final_state['question_clarified']}")
print(f"Resposta: {final_state['answer']}")

## Parte 2: O Workflow Condicional - O "Agente de Viagens Inteligente"

Agora, vamos ao nosso desafio principal: construir um agente que toma decisões. Este exercício irá introduzir o conceito de **Arestas Condicionais** e o padrão de projeto de usar **Chains** como ferramentas especializadas dentro dos nós.

### Fase 1: ✍️ Conceito e Arquitetura

**🎯 Missão:** Gerar um roteiro de viagem personalizado de 3 dias com base no destino e no tipo de viagem (ex: "praia" ou "aventura") fornecidos pelo usuário.

**🗺️ Fluxograma:**
![Fluxograma do Agente de Viagens](https://i.imgur.com/g7vOQ0Q.png)

> **Nota de Arquitetura:** O ponto chave aqui é a **decisão**. O agente não segue um caminho único. Ele analisa a entrada e **roteia** o trabalho para o especialista correto. 

### Fase 2: 🏛️ Estrutura (Ferramentas e Estado)

Seguindo as boas práticas, primeiro construiremos nossa **ferramenta especializada** – a `Chain` para gerar roteiros – e a testaremos de forma isolada. Depois, definiremos o `AgentState`.

In [ ]:
import json
from typing import List

from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.runnables import Runnable

# --- 2.1: Definição da Ferramenta (Itinerary Chain) ---

# Passo A: Definir a estrutura de saída com Pydantic
class Itinerary(BaseModel):
    """Define a estrutura de um roteiro de viagem."""
    roteiro: List[str] = Field(description="Uma lista contendo o roteiro de 3 dias.")

# Passo B: Configurar o Parser
parser = JsonOutputParser(pydantic_object=Itinerary)

# Passo C: Criar o Prompt
format_instructions = parser.get_format_instructions()
prompt_template_str = """Crie um roteiro de viagem de 3 dias para o local: {local}.
O tema da viagem é: {tema}.
A resposta deve ser um roteiro conciso e direto.
{format_instructions}
"""
prompt = PromptTemplate(
    template=prompt_template_str,
    input_variables=["local", "tema"],
    partial_variables={"format_instructions": format_instructions}
)

# Passo D: Montar a Chain com LCEL (LangChain Expression Language)
itinerary_chain: Runnable = prompt | llm | parser

# --- Teste Isolado da Ferramenta ---
print("--- Testando a itinerary_chain ---")
user_input_test = {
    "local": "Kyoto, Japão",
    "tema": "templos históricos e cultura tradicional",
}
response_test = itinerary_chain.invoke(user_input_test)

print("Conteúdo da resposta (formatado como JSON):")
print(json.dumps(response_test, indent=2, ensure_ascii=False))

In [ ]:
# --- 2.2: Modelagem do Estado ---

class AgentStateConditional(TypedDict):
    """Define a memória compartilhada para o Agente de Viagens."""
    local: str
    tipo_viagem: str
    roteiro: List[str]
    # Campo para guardar a decisão do roteador
    route: str

### Fase 3: ⚙️ Implementação (Nós e Grafo)

Com nossa ferramenta (`itinerary_chain`) e nosso estado (`AgentStateConditional`) prontos, podemos construir os nós.

In [ ]:
# --- 3.1: Codificação dos Nós ---

def classificar_viagem_node(state: AgentStateConditional) -> dict:
    """Nó Roteador: Analisa o tipo de viagem e atualiza o estado com a rota."""
    print("\n--- [NÓ DE DECISÃO: Classificando Tipo de Viagem] ---")
    tipo_viagem = state["tipo_viagem"].lower()

    if "praia" in tipo_viagem:
        print("  > Decisão: Roteiro de Praia.")
        return {"route": "gerar_roteiro_praia"}
    
    print("  > Decisão: Roteiro de Aventura.")
    return {"route": "gerar_roteiro_aventura"}


def gerar_roteiro_praia_node(state: AgentStateConditional) -> dict:
    """Nó Especialista: Gera um roteiro de praia usando a itinerary_chain."""
    print("\n--- [NÓ ESPECIALISTA: Gerando Roteiro de Praia] ---")
    tema = "atividades relaxantes na praia e vida noturna"
    local = state["local"]
    
    print(f"  > Usando a chain para o local: '{local}' com o tema '{tema}'")
    resultado_chain = itinerary_chain.invoke({"local": local, "tema": tema})
    
    return {"roteiro": resultado_chain["roteiro"]}


def gerar_roteiro_aventura_node(state: AgentStateConditional) -> dict:
    """Nó Especialista: Gera um roteiro de aventura usando a itinerary_chain."""
    print("\n--- [NÓ ESPECIALISTA: Gerando Roteiro de Aventura] ---")
    tema = "trilhas, montanhismo e exploração da natureza"
    local = state["local"]
    
    print(f"  > Usando a chain para o local: '{local}' com o tema '{tema}'")
    resultado_chain = itinerary_chain.invoke({"local": local, "tema": tema})
    
    return {"roteiro": resultado_chain["roteiro"]}

In [ ]:
# --- 3.2: Montagem do Grafo ---

workflow_conditional = StateGraph(AgentStateConditional)

# Adiciona os nós
workflow_conditional.add_node("classificar_viagem", classificar_viagem_node)
workflow_conditional.add_node("gerar_roteiro_praia", gerar_roteiro_praia_node)
workflow_conditional.add_node("gerar_roteiro_aventura", gerar_roteiro_aventura_node)

# Define o ponto de entrada
workflow_conditional.set_entry_point("classificar_viagem")

# Define a Aresta Condicional
workflow_conditional.add_conditional_edges(
    "classificar_viagem",
    lambda state: state["route"], # Lê a rota do estado para decidir
    {
        "gerar_roteiro_praia": "gerar_roteiro_praia",
        "gerar_roteiro_aventura": "gerar_roteiro_aventura"
    }
)

# Define as arestas de saída
workflow_conditional.add_edge("gerar_roteiro_praia", END)
workflow_conditional.add_edge("gerar_roteiro_aventura", END)

# Compila o grafo
agent_conditional = workflow_conditional.compile()

### Fase 4: ✅ Validação (Execução e Análise)

Com o agente montado, vamos visualizar sua arquitetura e testar os dois caminhos condicionais.

In [ ]:
# --- 4.1: Visualização do Grafo ---
print("Arquitetura do Agente de Viagens (Condicional):")
display(Image(agent_conditional.get_graph().draw_mermaid_png()))

In [ ]:
# --- 4.2: Execução e Teste ---

# Função auxiliar para imprimir os resultados de forma bonita
def print_resultado(resultado: dict):
    print("\n" + "="*50)
    print("🗺️  Roteiro de Viagem Final")
    print(f"📍 Destino: {resultado['local']}")
    print(f"🎭 Tema: {resultado['tipo_viagem']}")
    print("-" * 50)
    # CORREÇÃO: Removido o 'Dia X:' extra do print
    for dia in resultado['roteiro']:
        print(f"👉 {dia}")
    print("="*50)

# Teste 1: Viagem de praia
print("\n--- Testando o Agente de Viagens (PRAIA) ---")
input_praia = {
    "local": "Porto de Galinhas, Brasil", 
    "tipo_viagem": "quero relaxar na praia", 
    "roteiro": []
}
resultado_praia = agent_conditional.invoke(input_praia)
print_resultado(resultado_praia)

# Teste 2: Viagem de aventura
print("\n--- Testando o Agente de Viagens (AVENTURA) ---")
input_aventura = {
    "local": "Chapada Diamantina, Brasil",
    "tipo_viagem": "muita aventura e trilhas",
    "roteiro": []
}
resultado_aventura = agent_conditional.invoke(input_aventura)
# CORREÇÃO DO BUG: Usando a variável 'resultado_aventura' para o print
print_resultado(resultado_aventura)